In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年6月8日

重み付きMAPEを算出する関数を実装

In [3]:
def calcWeightedMAPEscore(
    inputDF :pd.DataFrame,
    inputColumnDict :dict,
) -> float:
    """calcWeightedMAPEscore()
    
    重み付き平均MAPEを算出する関数

    Args:
        inputDF (pd.DataFrame) : 下記のようなテーブル構成
        |<関数名>|<コール回数>|<MAPE>|
        inputColumnDict (dict) : 上記のテーブル構成を前提に、次の辞書を構成する。{"funcName":<関数名>,"call":<コール回数>,"MAPE":<MAPE>,}

    Returns:
        float: 重み付き平均MAPE
    
    """
    _col_funcName :str = inputColumnDict["funcName"]
    _col_call :str = inputColumnDict["call"]
    _col_MAPE :str = inputColumnDict["MAPE"]

    _ser_funcName :pd.Series = inputDF[_col_funcName]
    _ser_call :pd.Series = inputDF[_col_call]
    _ser_MAPE :pd.Series = inputDF[_col_MAPE]

    _ret_numerator :float = 0
    _ret_denominator :float = sum(_ser_call)

    for index in range(len(_ser_funcName)):
        _ret_numerator += _ser_call[index] * _ser_MAPE[index]

    return _ret_numerator / _ret_denominator

def test_calcWeightedMAPEscore():
    関数名 :list[str]= ["name0", "name1", "name2", "name3"]
    inputColumnDict = {"funcName":"関数名", "call":"コール回数", "MAPE":"MAPE"}
    # テストケース1
    コール回数 :list[float] = [1, 2, 3, 4]
    MAPE :list[float] = [1, 2, 3, 4]
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"関数名":関数名, "コール回数":コール回数, "MAPE":MAPE})
    expected_result = 3.0
    actually_result = calcWeightedMAPEscore(inputDF = inputDF, inputColumnDict=inputColumnDict)
    # print(inputDF)
    assert expected_result == actually_result, f"expected_result={expected_result},actually_result={actually_result}"
    # テストケース2
    コール回数 :list[float] = [5,4,3,2]
    MAPE :list[float] = [7,7,7,7]
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"関数名":関数名, "コール回数":コール回数, "MAPE":MAPE})
    expected_result = 7.0
    actually_result = calcWeightedMAPEscore(inputDF = inputDF, inputColumnDict=inputColumnDict)
    # print(inputDF)
    assert expected_result == actually_result, f"expected_result={expected_result},actually_result={actually_result}"
    # テストケース3
    コール回数 :list[float] = [1,2,3,4]
    MAPE :list[float] = [5,4,3,2]
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"関数名":関数名, "コール回数":コール回数, "MAPE":MAPE})
    expected_result = 3.0
    actually_result = calcWeightedMAPEscore(inputDF = inputDF, inputColumnDict=inputColumnDict)
    # print(inputDF)
    assert expected_result == actually_result, f"expected_result={expected_result},actually_result={actually_result}"


In [4]:
test_calcWeightedMAPEscore()

最適モデルを返す関数を実装

In [5]:
def return_bestModelObject(
    inputDF :pd.DataFrame,
    list_expVar :list[str],
    list_resVar :list[str],
    list_modelName :list[str]
):
    """return_bestModelObject()

    入力に対して最適なモデルを返す関数

    Args:
        inputDF (pd.DataFrame) : 下記のようなテーブル構成
        |<説明変数1>|<説明変数2>|...|<目的変数>|
        list_expVar (list[str]) : 説明変数のリスト
        list_resVar (list[str]) : 目的変数のリスト
        list_modelName (list[str]) : モデル名のリスト

    Returns:
        dict : {"object":各モデルのオブジェクト, "modelName":モデル名}

    """
    return {"object":None, "modelName":None}

def test_return_bestModelObject():

    exp_1 :np.ndarray = np.linspace(0, 10, 10)
    exp_2 :np.ndarray = np.linspace(10, 0, 10)
    exp_3 :np.ndarray = np.linspace(-10, 10, 10)

    coefficient_1 :int = 7
    coefficient_2 :int = 5
    coefficient_3 :int = -3

    list_modelName :list[str] = ["modelLin", "modelIp", "modelLog"]
    list_expVar :list[str] = ["exp_1", "exp_2", "exp_3"]
    list_resVar :list[str] = ["res_"]

    # 線形モデル
    res_ :np.ndarray = coefficient_1 * exp_1 + coefficient_2 * exp_2 + coefficient_3 * exp_3
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"exp_1":exp_1, "exp_2":exp_2, "exp_3":exp_3, "res_":res_})
    retDict = return_bestModelObject(inputDF=inputDF, list_expVar=list_expVar, list_resVar=list_resVar, list_modelName=list_modelName)

    expected :str = "modelLin"
    actually :str = retDict["modelName"]
    assert actually == expected, f"expected={expected}, actually={actually}"

    # 反比例モデル
    res_ :np.ndarray = coefficient_1 / exp_1 + coefficient_2 / exp_2 + coefficient_3 / exp_3
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"exp_1":exp_1, "exp_2":exp_2, "exp_3":exp_3, "res_":res_})
    retDict = return_bestModelObject(inputDF=inputDF, list_expVar=list_expVar, list_resVar=list_resVar, list_modelName=list_modelName)

    expected :str = "modelIp"
    actually :str = retDict["modelName"]
    assert actually == expected, f"expected={expected}, actually={actually}"

    # 対数モデル
    res_ :np.ndarray = coefficient_1 * np.log10(exp_1) + coefficient_2 * np.log10(exp_2) + coefficient_3 * np.log10(exp_3)
    inputDF :pd.DataFrame = pd.DataFrame.from_dict({"exp_1":exp_1, "exp_2":exp_2, "exp_3":exp_3, "res_":res_})
    retDict = return_bestModelObject(inputDF=inputDF, list_expVar=list_expVar, list_resVar=list_resVar, list_modelName=list_modelName)

    expected :str = "modelLog"
    actually :str = retDict["modelName"]
    assert actually == expected, f"expected={expected}, actually={actually}"
    
    return 0


In [6]:
test_return_bestModelObject()

AssertionError: expected=modelLin, actually=None

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 
